#SocialAL Model
#generate trial-by-trial output from model fits

KLS 2.7.20
Project info: https://osf.io/b48n2/

Model modified from : Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

Python version and modules

In [1]:
import sys
print(sys.version)  
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import os
from decimal import *

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


Define Functions

In [2]:
def update_value(Prob):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
    EV = [(retain[x] + Prob*shared[x]) for x in range(0,4)]
    return EV

def update_prob(recip, Prob, a_gain, a_loss):
    gain = max(recip - Prob, 0)
    loss = min(recip - Prob, 0)
    Prob = Prob + a_gain * gain + a_loss * loss
    return Prob

def get_action_selection_prob(beta, EV, choice):
    actionProb = Decimal(np.exp(beta*EV[choice-1])/np.sum([np.exp(beta*x) for x in EV]))
    return actionProb

def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

def get_likelihood_action(params, data):
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    decay = params[3]
    
    # initialize variables
    prob = [0.5, 0.5, 0.5]
    ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    tletters = ["A", "B", "C"]
    
    totalLLH = 0 
    for trial in range(0, len(data)):

        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        if choice != 0:
            # use the probability of the selection (choice-probability) to update log likelihood
            cprob = probs[choice-1] #print(cprob, isinstance(cprob, float))
           
            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
            

            # update prob and value
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], a_gain, a_loss)
            ev[trustee] = update_value(prob[trustee])
                        
            # decay prob for other possible trustees - added 8.31.20
            options = [0, 1, 2]
            if choice != 1:
                options.pop(trustee)
            for x in options:
                prob[x] = prob[x] + decay * (0.5 - prob[x])
        
        print("Trial #: ", trial + 1)
        print("Trustee: ", tletters[trustee])
        print('EV for Trustee:', ev[trustee])
        print('choice prob: ', cprob)
        #print('Probs after decay: ', prob)
        #print('LLH: ', totalLLH)
        #print('Trustee Prob: ', prob[trustee])
        #print('Trustee Expected values: ', ev[trustee])
        
        
    return totalLLH

def model_fit_once(data, param):
        # initialize free parameters with randomly chosen numbers
        a_gain=param[0]
        a_loss=param[1]
        beta=param[2]
        decay = param[3]
        params = [a_gain, a_loss, beta, decay]
        
        #results = minimize(get_likelihood_action, 
                           #params, args =(data), method='BFGS', options = {'maxiter': 10000, 'disp': False})
        results = minimize(get_likelihood_action, 
                       params, args =(data), bounds = [(0, 1), (0, 1), (1e-10, 100), (0, 1)], 
                       options = {'maxiter': 10000, 'disp': False})
        return results

Get data

In [3]:
dt = pd.read_csv('../data/modeling/sub-1004.csv')
#dt = pd.read_csv('../data/modeling/sub-1008.csv') #- try getting this sub to work
#dt = pd.read_csv('../data/modeling/sub-2032.csv') #- try getting this sub to work
# recode trial type into numbers for model
def stims(trial_type):
    if trial_type == "Trustworthy":
        return 0
    elif trial_type == "Neutral":
        return 1
    elif trial_type == "Untrustworthy":
        return 2
dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
# rename response_key to choice
def choices(response_key):
    if response_key == 'None':
        return 0 
    else:
        return response_key  
dt['Choice'] = dt['response_key'].apply(choices)
dt['Choice'] = pd.to_numeric(dt['Choice'])
# calculte the trustee response
def resp(trial_earnings):
    if trial_earnings >= 12:
        return 1
    else:
        return 0
dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
#print(data)

Input parameters

In [4]:
array = [ 0.2988556 ,  0.33844715,  0.65910056,  0.05564249, 41.97350632]


In [5]:
model_fit_once(data, array)

Trial #:  1
Trustee:  A
EV for Trustee: [9.0, 9.8965668, 10.7931336, 11.6897004]
choice prob:  0.25
Trial #:  2
Trustee:  C
EV for Trustee: [9.0, 7.984658550000001, 6.9693171000000005, 5.95397565]
choice prob:  0.25
Trial #:  3
Trustee:  A
EV for Trustee: [9.0, 10.490211453681376, 11.980422907362751, 13.470634361044125]
choice prob:  0.1510598241592304435965132825003820471465587615966796875
Trial #:  4
Trustee:  B
EV for Trustee: [9.0, 7.984658550000001, 6.9693171000000005, 5.95397565]
choice prob:  0.25
Trial #:  5
Trustee:  C
EV for Trustee: [9.0, 7.385627219283688, 5.771254438567374, 4.156881657851061]
choice prob:  0.1374033967806700118341467486970941536128520965576171875
Trial #:  6
Trustee:  B
EV for Trustee: [9.0, 9.22427777094651, 9.448555541893018, 9.672833312839527]
choice prob:  0.26830736820305844947398554722894914448261260986328125
Trial #:  7
Trustee:  B
EV for Trustee: [9.0, 10.053817903143628, 11.107635806287256, 12.161453709430882]
choice prob:  0.265537678522714759576

Trial #:  3
Trustee:  A
EV for Trustee: [9.0, 10.490224754605965, 11.980449509211928, 13.470674263817893]
choice prob:  0.151054276825168976028379574927384965121746063232421875
Trial #:  4
Trustee:  B
EV for Trustee: [9.0, 7.984635102089721, 6.96927020417944, 5.95390530626916]
choice prob:  0.25
Trial #:  5
Trustee:  C
EV for Trustee: [9.0, 7.385659212980903, 5.771318425961804, 4.156977638942706]
choice prob:  0.1373987736978739826465556461698724888265132904052734375
Trial #:  6
Trustee:  B
EV for Trustee: [9.0, 9.224339342490387, 9.448678684980774, 9.67301802747116]
choice prob:  0.268305587029534053922219527521519921720027923583984375
Trial #:  7
Trustee:  B
EV for Trustee: [9.0, 10.053890657069573, 11.107781314139148, 12.161671971208722]
choice prob:  0.26554130175313461759145639007329009473323822021484375
Trial #:  8
Trustee:  C
EV for Trustee: [9.0, 7.032316176923533, 5.064632353847067, 3.0969485307705997]
choice prob:  0.2292442288932149130342708076568669639527797698974609375
Tri

      fun: 41.973506319290706
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.77635684e-05, -1.35003120e-05, -2.27373675e-05,  1.56319402e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 15
      nit: 1
   status: 0
  success: True
        x: array([0.29885563, 0.33844717, 0.65910059, 0.05564263])